# Solving the Heat Equation with FEniCSx

## Introduction

The model problem for the time-dependent partial differential equation (PDE) reads:
$$
\begin{align}
    \frac{\partial u}{\partial t} &= \nabla^2 u + f && \text{in } \Omega \times (0, T], \\
    u &= u_D && \text{on } \partial\Omega \times (0, T], \\
    u &= u_0 && \text{at } t = 0.
\end{align}
$$

Here, $ u $ varies with space and time, e.g. $ u = u(x, y, t) $ if the spatial domain $ \Omega $ is two-dimensional. The source function $ f $ and the boundary values $ u_D $ may also vary with space and time. The initial condition $ u_0 $ is a function of space only.

For this problem, we will use the Dirichlet boundary condition $ u_D = 0 $.

Here, on the domain $ \Omega = (0, 2) \times (0, 2) $, we consider:
$$
\begin{aligned}
u_0(x) &= x_1 x_2 (x_1 -2)(x_2 -2), \\
f(x, t) &= -e^{-t}\left[ x_1 x_2 (x_1 -2)(x_2 -2) + 2 x_1 (x_1 -2) + 2 x_2 (x_2 -2) \right].
\end{aligned}
$$

## Weak Formulation

To derive the weak formulation, we multiply the PDE by a test function $ v \in V $ (where $ V $ is a suitable function space) and integrate over the domain $ \Omega $:
$$
\begin{aligned}
\int_{\Omega} \frac{\partial u}{\partial t} v \, \mathrm{d}x &= \int_{\Omega} \nabla^2 u v \, \mathrm{d}x + \int_{\Omega} f v \, \mathrm{d}x.
\end{aligned}
$$

Applying integration by parts to the second term on the right-hand side and assuming $ u = u_D = 0 $ on $ \partial \Omega $, we obtain:
$$
\begin{aligned}
\int_{\Omega} \frac{\partial u}{\partial t} v \, \mathrm{d}x + \int_{\Omega} \nabla u \cdot \nabla v \, \mathrm{d}x &= \int_{\Omega} f v \, \mathrm{d}x.
\end{aligned}
$$

## Time Discretization: Euler Backward Scheme

We discretize in time using the Euler backward scheme. Let $ u^n $ be the approximation of $ u $ at time $ t_n $. The time derivative can be approximated as:
$$
\frac{\partial u}{\partial t} \approx \frac{u^{n+1} - u^n}{\Delta t},
$$
where $ \Delta t $ is the time step size. Substituting this into the weak formulation gives:
$$
\begin{aligned}
\int_{\Omega} \frac{u^{n+1} - u^n}{\Delta t} v \, \mathrm{d}x + \int_{\Omega} \nabla u^{n+1} \cdot \nabla v \, \mathrm{d}x &= \int_{\Omega} f^{n+1} v \, \mathrm{d}x.
\end{aligned}
$$

Rearranging, we get the fully discrete form:
$$
\begin{aligned}
\int_{\Omega} u^{n+1} v \, \mathrm{d}x + \Delta t \int_{\Omega} \nabla u^{n+1} \cdot \nabla v \, \mathrm{d}x &= \int_{\Omega} u^n v \, \mathrm{d}x + \Delta t \int_{\Omega} f^{n+1} v \, \mathrm{d}x.
\end{aligned}
$$

## Spatial Discretization: Crouzeix-Raviart Finite Element Method

For spatial discretization, we use the Crouzeix-Raviart finite element method, which is a nonconforming finite element method suitable for solving the heat equation. The Crouzeix-Raviart element is defined on triangles and has degrees of freedom at the midpoints of edges. 

Let $V_h$ be the finite element space consisting of Crouzeix-Raviart elements. For the sack of simplicity,  we consider the same $V_h$ over time. The discrete solution $ u_h \in V_h $ satisfies:
$$
\begin{aligned}
\int_{\Omega} u_h^{n+1} v_h \, \mathrm{d}x + \Delta t \int_{\Omega} \nabla u_h^{n+1} \cdot \nabla v_h \, \mathrm{d}x &= \int_{\Omega} u_h^n v_h \, \mathrm{d}x + \Delta t \int_{\Omega} f^{n+1} v_h \, \mathrm{d}x \quad \forall v_h \in V_h.
\end{aligned}
$$


This completes the weak formulation and discretization of the dynamic heat equation using the Euler backward scheme in time and the Crouzeix-Raviart finite element method in space.

# Implementation

In [1]:
import matplotlib as mpl
import pyvista
import ufl
import numpy as np
from ufl import SpatialCoordinate
from dolfinx import cpp as _cpp

from petsc4py import PETSc
from mpi4py import MPI

from dolfinx import fem, mesh, io, plot
from dolfinx.fem.petsc import assemble_vector, assemble_matrix, create_vector, apply_lifting, set_bc

In [2]:
# Create the mesh and define the function space

nx, ny = 128, 128
domain = mesh.create_rectangle(MPI.COMM_WORLD, [np.array([0,0]), np.array([2,2])],
                               [nx, ny], mesh.CellType.triangle
                               )

V = fem.functionspace(domain, ("CR", 1))

In [3]:
# Determine the dimension of the boundary facets (one less than the dimension of the domain)
fdim = domain.topology.dim -2

# Locate the boundary facets of the mesh
# The lambda function selects all facets onassemble_matrix the boundary by returning 'True' for all points
boundary_facets = mesh.locate_entities_boundary(
    domain, fdim, lambda x: np.full(x.shape[1], True, dtype=bool)
)

# Create a Dirichlet boundary condition
# Set the value to 0 on the located boundary facets for the function space 'V'
bc = fem.dirichletbc(PETSc.ScalarType(0), fem.locate_dofs_topological(V, fdim, boundary_facets), V)


In [4]:
# Define the time-dependent source term f(x, y, t)
class TimeDependentSource:
    def __init__(self, t=0):
        self.t = t

    def eval(self, x):
        return -(x[0]*x[1]*(x[0]-2)*(x[1]-2)+2*x[0]*(x[0]-2)+2*x[1]*(x[1]-2)) * np.exp(-self.t)

f_expr = TimeDependentSource()

# Define the initial condition u(x, y, 0) 
def initial_condition(x):
    return x[0]*x[1]*(x[0]-2)*(x[1]-2)#np.exp(-5 * (x[0]**2 + x[1]**2))

u_n = fem.Function(V)
u_n.name = "u_n"
u_n.interpolate(initial_condition)

In [5]:
# Define temporal parameters
t = 0 #Start time
T = 3.0#Final time
num_steps= 200
dt = T / num_steps

In [6]:
# Define the trial and test functions
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

In [7]:
# Define the variational problem
f = fem.Function(V)
f.interpolate(f_expr.eval)

a = u * v *ufl.dx + dt * ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = (u_n + dt * f) * v * ufl.dx

In [8]:
# Assemble the linear system
A = assemble_matrix(fem.form(a))
A.assemble()
b = create_vector(fem.form(L))

In [9]:
# xdmf = io.XDMFFile(domain.comm, "solution_CR.xdmf", "w")
# xdmf.write_mesh(domain)

# Define solution variable, and interpolate initial solution for visualization in Paraview
u_sol = fem.Function(V)
u_sol.name = "u_sol"
u_sol.interpolate(initial_condition)
# xdmf.write_function(u_sol, t)

In [10]:
# Get the topology of the mesh

tdim = domain.topology.dim
entities = range(domain.topology.index_map(tdim).size_local)
dofmap = V.dofmap


num_dofs_per_cell = V.dofmap.dof_layout.num_dofs
cell_type = domain.topology.cell_type
perm = np.argsort(_cpp.io.perm_vtk(cell_type, num_dofs_per_cell))
cell_types = np.full(domain.topology.index_map(tdim).size_local, mesh.CellType.triangle, dtype=np.uint8)

topology = np.zeros((len(entities), num_dofs_per_cell + 1), dtype=np.int32)
topology[:, 0] = num_dofs_per_cell

dofmap_ = dofmap.list
#
topology[:, 1:] = dofmap_[: len(entities), perm]
#
cells = topology.reshape(1, -1)[0]
#
geometry = V.tabulate_dof_coordinates()

In [11]:
#Visualization with pyvista

pyvista.start_xvfb()

grid = pyvista.UnstructuredGrid(cells, cell_types, geometry)

plotter = pyvista.Plotter()
plotter.open_gif("u_sol.gif", fps=15)

grid.point_data["u_sol"] = u_sol.x.array
warped = grid.warp_by_scalar("u_sol", factor=1)

viridis = mpl.colormaps.get_cmap("viridis").resampled(25)
sargs = dict(title_font_size=25, label_font_size=20, fmt="%.2f", color="black",
             position_x=0.1, position_y=0.8, width=0.8, height=0.1)

# plotter.add_mesh(grid, color="red", show_edges=True, edge_color="green", opacity=0.5, label="Original Grid")
renderer = plotter.add_mesh(warped, show_edges=True, edge_color="blue", lighting=False,
                            cmap=viridis, scalar_bar_args=sargs,
                            clim=[0, max(u_sol.x.array)])

In [12]:
# Create the linear solver
solver = PETSc.KSP().create(domain.comm)
solver.setOperators(A)
solver.setType(PETSc.KSP.Type.PREONLY)
solver.getPC().setType(PETSc.PC.Type.LU)

In [ ]:
while t < T:
    t += dt
    f_expr.t = t
    f.interpolate(f_expr.eval)

    with b.localForm() as loc_b:
        loc_b.set(0)

    assemble_vector(b, fem.form(L))
    apply_lifting(b, [fem.form(a)], [[bc]])#, x0=[u_sol.vector]
    b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    set_bc(b, [bc])

    solver.solve(b, u_sol.vector)
    u_sol.x.scatter_forward()

    # Update the solution for the next time step
    u_n.x.array[:] = u_sol.x.array[:]

    # Optionally, save the solution or visualize it
    # e.g., save to a file or plot using matplotlib
    
    # Write solution to file
    # xdmf.write_function(u_sol, t)
    # Update plot
    new_warped = grid.warp_by_scalar("u_sol", factor=1)
    warped.points[:, :] = new_warped.points
    warped.point_data["u_sol"][:] = u_sol.x.array
    plotter.write_frame()
plotter.close()
# xdmf.close()

print("Simulation complete.")

<img src="./u_sol.gif" alt="gif" width="800px">

**References**
 - [Fenicsx tutorials](\href{https://jsdokken.com/dolfinx-tutorial/})